In [61]:
import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit

import warnings
warnings.filterwarnings('ignore')

In [62]:
# Import data
obs_A = pd.read_parquet('../../preprocessing/data/obs_A.parquet')
obs_B = pd.read_parquet('../../preprocessing/data/obs_B.parquet')
obs_C = pd.read_parquet('../../preprocessing/data/obs_C.parquet')
est_A = pd.read_parquet('../../preprocessing/data/est_A.parquet')
est_B = pd.read_parquet('../../preprocessing/data/est_B.parquet')
est_C = pd.read_parquet('../../preprocessing/data/est_C.parquet')
test_A = pd.read_parquet('../../preprocessing/data/test_A.parquet')
test_B = pd.read_parquet('../../preprocessing/data/test_B.parquet')
test_C = pd.read_parquet('../../preprocessing/data/test_C.parquet')

# Columns to drop
columns = [
    'date_forecast',
    'super_cooled_liquid_water:kgm2',
    'air_density_2m:kgm3',
    'snow_water:kgm2',
    'precip_5min:mm',
    'precip_type_5min:idx',
    'rain_water:kgm2',
    'snow_melt_10min:mm',
    'dew_or_rime:idx',
    'snow_depth:cm'
]

# Drop columns
obs_A = obs_A.drop(columns=columns)
obs_B = obs_B.drop(columns=columns)
obs_C = obs_C.drop(columns=columns)
est_A = est_A.drop(columns=columns)
est_B = est_B.drop(columns=columns)
est_C = est_C.drop(columns=columns)
test_A = test_A.drop(columns=columns)
test_B = test_B.drop(columns=columns)
test_C = test_C.drop(columns=columns)

In [70]:
# Concatinate
A = pd.concat([obs_A, est_A])
B = pd.concat([obs_B, est_B])
C = pd.concat([obs_C, est_C])

# Data splits for submissions
# X_A = A.drop(columns='pv_measurement')
# y_A = A['pv_measurement']
# X_B = B.drop(columns='pv_measurement')
# y_B = B['pv_measurement']
# X_C = C.drop(columns='pv_measurement')
# y_C = C['pv_measurement']

# Data splits for testing
train_A, test_A = train_test_split(A, test_size=0.2, shuffle=True, random_state=42)
X_train_A = train_A.drop(columns='pv_measurement')
y_train_A = train_A['pv_measurement']
X_test_A = test_A.drop(columns='pv_measurement')
y_test_A = test_A['pv_measurement']

train_B, test_B = train_test_split(B, test_size=0.2, shuffle=True, random_state=42)
X_train_B = train_B.drop(columns='pv_measurement')
y_train_B = train_B['pv_measurement']
X_test_B = test_B.drop(columns='pv_measurement')
y_test_B = test_B['pv_measurement']

train_C, test_C = train_test_split(C, test_size=0.2, shuffle=True, random_state=42)
X_train_C = train_C.drop(columns='pv_measurement')
y_train_C = train_C['pv_measurement']
X_test_C = test_C.drop(columns='pv_measurement')
y_test_C = test_C['pv_measurement']



In [46]:
# Feature selection
# sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
# X_reduced = sel.fit_transform(X)
# print('Columns with low variance:')
# for cols in X.columns:
#     if cols not in X.columns[sel.get_support(indices=True)]:
#         print(cols)


# Select the k best features based on F-value for regression
k = 45  # You can choose k based on your requirements
selector = SelectKBest(f_regression, k=k)

# Fit the selector to your data
X_new = selector.fit_transform(X, y)

# Get the indices of the selected features
selected_indices = selector.get_support(indices=True)

# Get the selected features
selected_features = X.columns[selected_indices]

print("Selected features:", selected_features)

# Create a new DataFrame with only the selected features
X_selected = X[selected_features]
print(X_selected.shape)

Selected features: Index(['clear_sky_rad:W', 'clear_sky_energy_1h:J', 'diffuse_rad:W',
       'diffuse_rad_1h:J', 'direct_rad:W', 'direct_rad_1h:J',
       'effective_cloud_cover:p', 'sun_elevation:d',
       'absolute_humidity_2m:gm3', 't_1000hPa:K', 'total_cloud_cover:p',
       'visibility:m', 'msl_pressure:hPa', 'dew_point_2m:K',
       'relative_humidity_1000hPa:p', 'pressure_gradient',
       'temp_dewpoint_diff', 'total_radiation', 'is_day:idx',
       'is_in_shadow:idx', 'pressure_100m:hPa', 'pressure_50m:hPa',
       'sfc_pressure:hPa', 'prob_rime:p', 'date_forecast_fft_phase',
       't_1000hPa:K_rate_of_change', 'clear_sky_rad:W_rate_of_change',
       'clear_sky_rad:W_rate_of_change_of_change',
       'diffuse_rad:W_rate_of_change',
       'diffuse_rad:W_rate_of_change_of_change', 'direct_rad:W_rate_of_change',
       'direct_rad:W_rate_of_change_of_change',
       'total_radiation_rate_of_change',
       'total_radiation_rate_of_change_of_change', 'observed',
       'sun_a

In [47]:

model = RandomForestRegressor()

# Set up cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Set up SFS
sfs = SequentialFeatureSelector(model, n_features_to_select='auto', cv=tscv, scoring='neg_mean_absolute_error', n_jobs=-1)

# Fit SFS
X_transformed = sfs.fit_transform(X_selected, y)

# Get the selected feature indices
selected_features = sfs.get_support(indices=True)

print('Selected features:')
for features in selected_features:
    print(features)

KeyboardInterrupt: 

In [65]:
# Initalize the models
parameters = {'colsample_bytree': 0.664, 
              'gamma': 3, 
              'learning_rate': 0.012, 
              'max_depth': 12, 
              'min_child_weight': 15, 
              'n_estimators': 500, 
              'reg_alpha': 2, 
              'reg_lambda': 2, 
              'subsample': 0.912}

model_A = xgb.XGBRegressor(**parameters)
model_B = xgb.XGBRegressor(**parameters)
model_C = xgb.XGBRegressor(**parameters)

In [66]:
# Train prediction model
model_A.fit(X_A, y_A)
model_B.fit(X_B, y_B)
model_C.fit(X_C, y_C)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.664, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=3, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.012, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=12, max_leaves=None,
             min_child_weight=15, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [67]:
# Evaluate
print('MAE A:', mean_absolute_error(y_A, model_A.predict(X_A)))
print('MAE B:', mean_absolute_error(y_B, model_B.predict(X_B)))
print('MAE C:', mean_absolute_error(y_C, model_C.predict(X_C)))

MAE A: 71.29554349659497
MAE B: 8.66896795779669
MAE C: 8.08803972364211


In [68]:
# Feature importance
feature_importances = model_A.feature_importances_
feature_importances = pd.DataFrame({'feature': list(X_A.columns), 'importance': feature_importances}).sort_values('importance', ascending = False)

# Print feature importance
for i in range(feature_importances.shape[0]):
    print(f"{i} {feature_importances.iloc[i, 0]}: {feature_importances.iloc[i, 1]}")

0 total_radiation: 0.37481001019477844
1 direct_rad:W: 0.23175698518753052
2 direct_rad_1h:J: 0.11530543863773346
3 diffuse_rad:W: 0.02941821701824665
4 clear_sky_rad:W: 0.01888495683670044
5 hour: 0.016049012541770935
6 sun_azimuth:d_lag_7: 0.0109621686860919
7 sun_elevation:d: 0.010946950875222683
8 snow_accumulation: 0.009801385924220085
9 total_radiation_rolling_avg_3: 0.007572078611701727
10 effective_cloud_cover:p: 0.006856752093881369
11 relative_humidity_1000hPa:p_lag_-3: 0.005361482966691256
12 sun_azimuth:d: 0.00532489363104105
13 date_forecast_fft_amplitude: 0.004698095843195915
14 clear_sky_energy_1h:J: 0.004331118427217007
15 temp_dewpoint_diff_lag_-4: 0.0042458176612854
16 diffuse_rad_1h:J: 0.004193869419395924
17 clear_sky_rad:W_rate_of_change: 0.004029703326523304
18 t_1000hPa:K_rolling_avg_24: 0.003981053363531828
19 total_cloud_cover:p: 0.0038206882309168577
20 msl_pressure:hPa_lag_3: 0.003781562903895974
21 sfc_pressure:hPa: 0.003657728433609009
22 t_1000hPa:K_lag_4:

In [71]:
# Create submission

output_file = 'submission.csv'

pred_A = model_A.predict(test_A)
pred_B = model_B.predict(test_B)
pred_C = model_C.predict(test_C)

pred_A = np.clip(pred_A, 0, None)
pred_B = np.clip(pred_B, 0, None)
pred_C = np.clip(pred_C, 0, None)

# Concatenate predictions
predictions = np.concatenate([pred_A, pred_B, pred_C])

# Create an id array
ids = np.arange(0, len(predictions))

# Create a DataFrame
df = pd.DataFrame({
    'id': ids,
    'prediction': predictions
})

# Save to CSV
df.to_csv(output_file, index=False)
print(f"Submission saved to {output_file}")

Submission saved to submission.csv
